### Correction of the ML-Ready Cell-Painting Data Sets
#### Prerequisite
This notebook is part of the master thesis of Luis Vollmers. It uses the cell paintin data set of Bray et al. and the goal of the project is to predict pubchem assay data. Originally this notebook was concerned with pubchem AID 1030 but this might be subject to change.
#### Introduction
The datasets generated in ml_ready of step 01 comprise several mistakes that have been made and that need to be corrected within this jupyter notebook. Mostly, the compounds listed in the rows of the ML-ready dataframes are not unique, due to a misunderstanding stemming from the Bray et al Cell-Painting paper. It was stated that the Metadata_broad_sample is a unique identifier which was found to be wrong. Identical compounds which were used in different concentrations and/or on different well-plates were assigned different Metadata_broad_sample values. This needs to be corrected as well as the averaging over the different column values. In the data columns, the cell painting features are listed and so far the algorithm just kept the first concentration and took the average of the respective multiplicates. The correct way of doing it however is to check how many concentrations are present and then take the median of the concentration that is most frequent which is done by this jupyter notebook. 
#### Summary of the steps in this notebook
1. Import the ML-ready pubchem-assay and the preprocessed cell-painting raw data
2. Reintroduce the center median data into the pubchem df
3. Treat the multi-concentration Compounds adequately
4. Export the output


In [6]:
import pandas as pd
pd.set_option("max_rows",1200)

#### 1. Import the ML-ready pubchem-assay and the preprocessed cell-painting raw data
- Inputs are taken from the directory for step 1 of the pubchem assays and from the preprocessing directory
- the data from cp_1030 is erroneous but the meta data is needed for the overlap with the center median data
- therefore filter the df for the metadata

In [2]:
# load the data into RAM 'cp_1030' can be used as a variable for a bash script
dataset_path='../../01-FilteringAssays/ml_ready/cp_588852.csv'
df = pd.read_csv(dataset_path, index_col=0).drop(axis=1,columns=['Unnamed: 0.1'])

# read the cell-painting data into RAM
cp = pd.read_csv('../../../preprocessing/cp_center_median.csv', index_col=0)

# define the meta cols of the pubchem assays that are relevant and filter the df 
meta_cols = ['Metadata_broad_sample','PUBCHEM_ACTIVITY_OUTCOME', 'PUBCHEM_ACTIVITY_SCORE']
df = df.loc[:,meta_cols]

#### 2. Reintroduce the Center Median Data into the Pubchem DF
- the common column is Metadata_broad_sample which was responsible for the errors in the first place
- it is actually the only identifier column that is left in the ML-ready df
- the inner merge makes sure that only rows are kept that can be found in both dataframes

In [3]:
# merge the cp and the assay data on the common column
merged = pd.merge(left=cp, right=df, on='Metadata_broad_sample', how='inner')

In [4]:
# this cell calculates the number of rows expected from the final dataframe
multi_list = []
concs_list = merged.CAN_SMILES.value_counts().to_list()

for i in concs_list:
    if i > 1:
        multi_list.append(i)
        
merged.shape[0]-sum(multi_list)+len(multi_list)

8795

#### 3. Treat the multi-concentration Compounds adequately
- this step is the only one that needs to be conducted manually
- first look at the multi conc compounds and determine how to treat them
- in this case the concentrations that are less frequent are dropped and the remaining ones are calculated into median
- for that purpose the merged DF is split into data and meta columns and only the data columns are treated accordingly
- the groupby method is used to calculate the compound wise median 
- the single concentrations are already calculated so only the multi concs are actually computed herein

In [25]:
# have a look at the multi concentration compounds and figure out how to treat them
multi_merged = merged.query('SINGLE_CONC==False').loc[:,['CAN_SMILES','Metadata_mmoles_per_liter']]

for i,j in multi_merged.groupby("CAN_SMILES"):
    print("{}\n{}\n\n".format(i,j.Metadata_mmoles_per_liter.value_counts()))

CC(C)(C)c1ccc(C(O)CCCN2CCC(C(O)(c3ccccc3)c3ccccc3)CC2)cc1
2.120111    1
Name: Metadata_mmoles_per_liter, dtype: int64


CC(C)COC(=O)N(C)C[C@@H]1OCc2cn(nn2)CCCC(=O)N([C@@H](C)CO)C[C@H]1C
4.989704    1
Name: Metadata_mmoles_per_liter, dtype: int64


CC(C)COC(=O)N(C)C[C@@H]1OCc2cnnn2CCCC(=O)N([C@@H](C)CO)C[C@@H]1C
5.062167    1
Name: Metadata_mmoles_per_liter, dtype: int64


CC(C)COC(=O)N(C)C[C@@H]1OCc2cnnn2CCCC(=O)N([C@@H](C)CO)C[C@H]1C
5.095349    1
Name: Metadata_mmoles_per_liter, dtype: int64


CC(C)COC(=O)N(C)C[C@@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@@H]1C
5.225374    1
Name: Metadata_mmoles_per_liter, dtype: int64


CC(C)COC(=O)N(C)C[C@@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@H]1C
4.922948    1
Name: Metadata_mmoles_per_liter, dtype: int64


CC(C)COC(=O)N(C)C[C@H]1OCc2cn(nn2)CCCC(=O)N([C@@H](C)CO)C[C@@H]1C
5.032085    1
Name: Metadata_mmoles_per_liter, dtype: int64


CC(C)COC(=O)N(C)C[C@H]1OCc2cnnn2CCCC(=O)N([C@H](C)CO)C[C@@H]1C
4.981522    1
Name: Metadata_mmoles_per_liter, dtype: int

In [14]:
merged = merged.drop(merged[(merged.SINGLE_CONC==False) & (merged.Metadata_mmoles_per_liter > 4.0) & (merged.CAN_SMILES=='CC(C)(C)c1ccc(C(O)CCCN2CCC(C(O)(c3ccccc3)c3ccccc3)CC2)cc1')].index)
merged = merged.drop(merged[(merged.SINGLE_CONC==False) & (merged.Metadata_mmoles_per_liter > 4.0) & (merged.CAN_SMILES=='COC(=O)Nc1nc2cc(SC(C)C)ccc2[nH]1')].index)
merged = merged.drop(merged[(merged.SINGLE_CONC==False) & (merged.Metadata_mmoles_per_liter > 4.0) & (merged.CAN_SMILES=='C[C@@H]1[C@H]2C3=CC[C@@H]4[C@@]5(C)CC[C@H](O)C(C)(C)[C@@H]5CC[C@@]4(C)[C@]3(C)CC[C@@]2(C(=O)O)CC[C@H]1C')].index)
merged = merged.drop(merged[(merged.SINGLE_CONC==False) & (merged.Metadata_mmoles_per_liter > 4.0) & (merged.CAN_SMILES=='Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21')].index)

merged.shape[0]

9832

In [15]:
# Quality Control step that makes sure only the relevant rows with the most concentrations are kept
merged.query('SINGLE_CONC==False').loc[:,['CAN_SMILES','Metadata_mmoles_per_liter']] # just needed as a one time lookup

,CAN_SMILES,Metadata_mmoles_per_liter
8653,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
8654,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
8655,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
8656,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
8657,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
8658,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
8659,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
8660,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
8661,CC(C)(C)c1ccc(C(O)CCCN2CCC(C(O)(c3ccccc3)c3ccc...,2.120111
8662,CC(C)(C)c1ccc(C(O)CCCN2CCC(C(O)(c3ccccc3)c3ccc...,2.120111


In [16]:
# define a list of the all columns of the merged data frame
all_cols = merged.columns.to_list()
# redefine meta columns with the meta information of the cell painting assay
meta_cols = ['CAN_SMILES','CPD_SMILES','Metadata_broad_sample','Metadata_Plate_Map_Name','Metadata_ASSAY_WELL_ROLE','Metadata_Plate','SINGLE_CONC','PUBCHEM_ACTIVITY_SCORE','PUBCHEM_ACTIVITY_OUTCOME']

In [17]:
# data cols are basically all columns without the meta data. hence the forloop that removes those from data_cols
data_cols = all_cols
for item in meta_cols:
    data_cols.remove(item)
    
# afterwards the 'CAN_SMILES' column is inserted at the first position
data_cols.insert(0,'CAN_SMILES')

In [18]:
# the merged data frame is split into two dataframes containing meta and raw data information
merged_data = merged.loc[:,data_cols]
merged_meta = merged.loc[:,meta_cols]

In [19]:
# this command takes the compound wise median of the data
merged_data = merged_data.groupby('CAN_SMILES').median().reset_index()

#### 4. Export the Output
- as a last step the only thing that needs to be done is to merge the meta and data columns back into one DF
- a bit of a clean up needs to be done since the merge command creates suplicates, which can be safely deleted
- output in csv format named according to the pubchem AID

In [20]:
# the median data is merged back with the meta data
merged = pd.merge(left=merged_meta, right=merged_data, on='CAN_SMILES', how='left')

In [21]:
# merging generally keeps all rows in both frames so that duplicates are generated, which get hereby deleted
merged = merged.drop_duplicates(subset='CAN_SMILES')

In [22]:
merged.to_csv('../_output/cp_588852.csv',index=False)

In [23]:
pd.read_csv('../_output/cp_588852.csv') # only uncomment for quality control purposes, i.e. visual conformation

,CAN_SMILES,CPD_SMILES,Metadata_broad_sample,Metadata_Plate_Map_Name,Metadata_ASSAY_WELL_ROLE,Metadata_Plate,SINGLE_CONC,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_OUTCOME,Metadata_mmoles_per_liter,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0
0,COc1cc(OC)c2ccc(=O)oc2c1,COc1cc(OC)c2ccc(=O)oc2c1,BRD-K78612426-001-02-6,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,-0.061549,-0.133824,-0.130796,-0.131496,-0.153068,-0.156526,-0.125545,0.107378,0.023594,0.045900
1,COC(=O)c1ccccc1NC(=O)c1ccc(F)cc1,COC(=O)c1ccccc1NC(=O)c1ccc(F)cc1,BRD-K78633253-001-01-2,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,0.098785,-0.029717,-0.036163,-0.049478,-0.122305,-0.091394,-0.115040,0.200919,0.124473,0.153696
2,O=C(CCCN1CCC(O)(c2ccc(Br)cc2)CC1)c1ccc(F)cc1,OC1(CCN(CCCC(=O)c2ccc(F)cc2)CC1)c1ccc(Br)cc1,BRD-K78643075-001-03-3,H-BIOA-007-3,treated,24278,True,0.0,Inactive,2.379167,...,0.208695,-0.024429,-0.044756,-0.048945,-0.091941,-0.096263,-0.109370,0.053506,0.038483,0.040109
3,Cc1oncc1C(=O)Nc1ccc(C(F)(F)F)cc1,Cc1oncc1C(=O)Nc1ccc(cc1)C(F)(F)F,BRD-K78692225-001-11-2,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,0.078186,-0.005418,-0.038088,-0.036953,-0.075759,-0.110328,-0.092795,0.107877,0.034836,0.063032
4,CCCCC1(COC(=O)CCC(=O)O)C(=O)N(c2ccccc2)N(c2ccc...,CCCCC1(COC(=O)CCC(O)=O)C(=O)N(N(C1=O)c1ccccc1)...,BRD-K78815826-001-04-7,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,-0.049773,-0.120946,-0.057446,-0.074382,-0.102800,-0.090232,-0.095308,0.124821,0.014721,0.040721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8790,CC(C)COC(=O)N(C)C[C@@H]1OCc2cnnn2CCCC(=O)N([C@...,CC(C)COC(=O)N(C)C[C@@H]1OCc2cnnn2CCCC(=O)N(C[C...,BRD-K63778487-001-02-8,H-CBLD-003-4,treated,24740,False,0.0,Inactive,5.062167,...,-0.107988,-0.067696,-0.106654,-0.078256,-0.038353,-0.034741,-0.035673,-0.004637,-0.003541,-0.009770
8791,C[C@H]1CN([C@H](C)CO)C(=O)CCCn2nncc2CO[C@@H]1C...,C[C@H](CO)N1C[C@H](C)[C@@H](CN(C)Cc2ccc3cccnc3...,BRD-K16670438-001-02-5,H-CBLD-003-4,treated,24740,False,0.0,Inactive,5.113747,...,-0.271176,-0.018774,-0.137886,-0.113983,-0.061528,-0.082561,-0.073975,-0.117175,-0.100076,-0.087587
8792,C[C@@H]1CN([C@H](C)CO)C(=O)CCCn2nncc2CO[C@H]1C...,C[C@H](CO)N1C[C@@H](C)[C@H](CN(C)C(=O)c2ccncc2...,BRD-K36882679-001-02-0,H-CBLD-003-4,treated,24740,False,0.0,Inactive,5.049401,...,0.022377,-0.033583,0.021928,0.004868,0.013618,0.035613,0.017948,0.050071,0.026995,0.040520
8793,C[C@@H](CO)N1C[C@@H](C)[C@@H](CN(C)C(=O)c2ccnc...,C[C@@H](CO)N1C[C@@H](C)[C@@H](CN(C)C(=O)c2ccnc...,BRD-K19662956-001-02-1,H-CBLD-003-4,treated,24740,False,12.0,Inactive,5.025291,...,-0.049191,-0.046927,-0.009726,-0.001119,-0.044611,-0.062832,-0.094449,0.034307,0.006336,0.011479
